### INGESTION

In [1]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime

In [2]:
#############Load config.json and get input and output paths
with open('config.json','r') as f:
    config = json.load(f) 

input_folder_path = config['input_folder_path']
print(input_folder_path)
output_folder_path = config['output_folder_path']
print(output_folder_path)

practicedata
ingesteddata


In [19]:
main_df = pd.DataFrame(columns=[
    'corporation', 'lastmonth_activity', 'lastyear_activity', 'number_of_employees', 'exited'
    ])
main_df.head(1)

,corporation,lastmonth_activity,lastyear_activity,number_of_employees,exited


In [27]:
ingested_files = []

In [28]:
current_dir = os.getcwd()
input_dir = os.path.join(os.getcwd(), input_folder_path)
input_files = os.listdir(input_dir)
for file_name in input_files:
    print(file_name)
    if '.csv' in file_name:
        print(os.path.join(input_dir,file_name))
        df = pd.read_csv(os.path.join(input_dir,file_name))
        main_df = pd.concat([main_df, df], ignore_index=True)
        ingested_files.append(file_name)

dataset2.csv
\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\practicedata\dataset2.csv
dataset1.csv
\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\practicedata\dataset1.csv


In [29]:
ingested_files

['dataset2.csv', 'dataset1.csv']

In [22]:
main_df.head()

,corporation,lastmonth_activity,lastyear_activity,number_of_employees,exited
0,abcd,78,1024,12,1
1,asdf,14,2145,20,0
2,xyzz,182,3891,35,0
3,acme,101,10983,2,1
4,qwer,0,118,42,1


In [23]:
print(main_df.shape)
main_df = main_df.drop_duplicates()
print(main_df.shape)

(36, 5)
(26, 5)


In [24]:
# save to outout folder
output_dir = os.path.join(os.getcwd(), output_folder_path)
print(output_dir)

\\wsl.localhost\Ubuntu\home\chafund\GIT\Dynamic_Risk_Assessment_System\ingesteddata


In [26]:
main_df.to_csv(os.path.join(output_dir, 'finaldata.csv'))

In [30]:
with open(os.path.join(output_dir, 'ingestedfiles.txt'), 'w') as tfile:
    tfile.write('\n'.join(ingested_files))